# Cavity Length Determiner

Use this notebook to find peak positions in a spectrum and use the free spectral range, along with the known or calculated material refractive index, to determine the cavity length in vacuum. The *must* be done at 0-degree incidence.

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import signal
from ruamel_yaml import YAML
import data_io
import material_properties as mp
import pmath

yaml = YAML()
sns.set_theme(context='notebook', style='whitegrid', palette='dark')
%matplotlib widget

In [ ]:
data_directory = 'path/to/angle/resolved/data/directory/'
angle_data, absorbance_data = data_io.get_angle_data_from_dir(data_directory)

print('Loading data from:')
print(data_directory, '\n')
print('Angles in range: [', angle_data[0][0], ',', angle_data[-1][0], ']')

In [ ]:
angle, wavenumber, intensity = angle_data[0]

lb = 500
ub = 8000
mask = (wavenumber >= lb) & (wavenumber <= ub)
wavenumber = wavenumber[mask]
intensity = intensity[mask]
peaks_in_interval = signal.find_peaks(intensity, height=1.0, distance=500)[0]
number_of_peaks = len(peaks_in_interval)
peak_pos_x = [wavenumber[p] for p in peaks_in_interval]
peak_pos_y = [intensity[p] for p in peaks_in_interval]
peaks_data = pd.DataFrame({'Peaks': peak_pos_x})

In [ ]:
# Plot the selected spectrum or a subrange
fig, ax = plt.subplots()

ax.plot(wavenumber, intensity)
ax.scatter(peak_pos_x, peak_pos_y, color='orange')

ax.set_title("Spectrum subrange at {} degrees\n{} peaks".format(angle, number_of_peaks))
ax.set_xlabel(r'Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Transmittance (%)')
plt.show()

In [ ]:
# Check the peak spacing statistics.
peaks_data.diff().describe()

In [ ]:
concentration = sample['Concentration']
solvent_refractive_index = 1.0 
solute_refractive_index = 1.0
n_eff = pmath.refractive_line(concentration, solvent_refractive_index, solute_refractive_index, 0.0, neat_solute_concentration)
L_cav = float(10**4 / (2 * n_eff * peaks_data.diff().mean()))

print('Concentration:', concentration, 'mol/L')
print('Refractive index:', n_eff)
print('Cavity length:', L_cav, 'micrometers')